In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from mvn.utils import multiview

from mvn.datasets.cmu import CMUSceneDataset
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset
from torch.utils.data import DataLoader

import os
from itertools import islice
from tqdm import tqdm_notebook
import pickle


import numpy as np
import cv2
import time
import sys

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from mvn.utils.img import image_batch_to_numpy, denormalize_image
from mvn.models.temporal import TemporalModel, TemporalModelBase
from mvn.models.basic import MVNetVolumetricHybridResNet_2, MVNetVolumetricHybridResNet_temporal
from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose

from mvn.utils import img
from mvn.utils import multiview
from mvn.utils import volumetric
from mvn.utils import op
from mvn.utils import vis
from mvn.utils import misc
from mvn.utils import cfg

from time import time

from easydict import EasyDict

from mvn.datasets import cmu, kth
from mvn.datasets import utils as dataset_utils
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset

from train import setup_human36m_dataloaders

from IPython.core.debugger import set_trace

from matplotlib import pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")

%matplotlib inline

retval = {
    'subject_names': ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11'],
    'camera_names': ['54138969', '55011271', '58860488', '60457274'],
    'action_names': [
        'Directions-1', 'Directions-2',
        'Discussion-1', 'Discussion-2',
        'Eating-1', 'Eating-2',
        'Greeting-1', 'Greeting-2',
        'Phoning-1', 'Phoning-2',
        'Posing-1', 'Posing-2',
        'Purchases-1', 'Purchases-2',
        'Sitting-1', 'Sitting-2',
        'SittingDown-1', 'SittingDown-2',
        'Smoking-1', 'Smoking-2',
        'TakingPhoto-1', 'TakingPhoto-2',
        'Waiting-1', 'Waiting-2',
        'Walking-1', 'Walking-2',
        'WalkingDog-1', 'WalkingDog-2',
        'WalkingTogether-1', 'WalkingTogether-2']
}
# import nonechucks

device = torch.cuda.current_device()

In [ ]:
# experiment_root = '../../logs/vanilla_v2v/h36_sv_dist_default_bs2-5@10.10.2019-17:58:03'
# config_path = experiment_root + '/config.yaml'

config_path = './experiments/paper/human36m/vol_2stage2/human36m_v2v_softmax.yaml'

config = cfg.load_config(config_path)

# config.opt.val_batch_size=1
# config.dataset.val.retain_every_n_frames_in_test = 1

train_loader, val_loader = setup_human36m_dataloaders(config)

In [ ]:
for batch in train_loader:
    break

In [ ]:
keypoints = batch['keypoints_3d']
volume_size = 64
cuboid_side = 2500
cuboids = []
sides = np.array([cuboid_side, cuboid_side, cuboid_side])

# keypoints = keypoints[:,1]

In [ ]:
 def get_coord_volumes_old(self, 
                            batch, 
                            cuboid_side, 
                            volume_size, 
                            device, 
                            determine_base_point = False, 
                            tri_keypoints_3d = None
                            ):
        '''
        determine_base_point (bool, optional): If set `True`, we determine it, using pre-calculated values, ground-truth or triangulation,
                                               if set `False`, we use default base point in the coodninate origin.              
        tri_keypoints_3d (torch.tensor, optional): keypoints given by triangulation, to extract base_point from them, 
                                                    determine_base_point should be True
        '''
        # build coord volumes


        cuboids = []
        batch_size = batch['keypoints_3d'].shape[0]
        coord_volumes = torch.zeros(batch_size, volume_size, volume_size, volume_size, 3, device=device)

        for batch_i in range(batch_size):

            # default base_point    
            base_point = np.array([0, 0, 0])      

            if determine_base_point:    
                if self.use_precalculated_pelvis or self.use_gt_pelvis:
                    keypoints_3d = batch['keypoints_3d'][batch_i] # [bs, dt, 17,4]
                else:
                    # here we have to determine pelvis by ourselves
                    keypoints_3d = tri_keypoints_3d[batch_i].detach().cpu().numpy()

                # get pelvis from keypoints   
                if self.use_separate_v2v_for_basepoint:
                    base_point = keypoints_3d[0, :3]
                else:         
                    if self.kind == "coco":
                        base_point = (keypoints_3d[11, :3] + keypoints_3d[12, :3]) / 2
                    elif self.kind == "mpii":
                        base_point = keypoints_3d[6, :3]

            # build cuboid
            sides = np.array([cuboid_side, cuboid_side, cuboid_side])
            # origin of the new coordinate system
            position = base_point - sides / 2
            cuboid = volumetric.Cuboid3D(position, sides)

            cuboids.append(cuboid)

            # build coord volume
            xxx, yyy, zzz = torch.meshgrid(torch.arange(volume_size, device=device),
                                            torch.arange(volume_size, device=device),
                                             torch.arange(volume_size, device=device))

            grid = torch.stack([xxx, yyy, zzz], dim=-1).type(torch.float)
            grid = grid.reshape((-1, 3))

            grid_coord = torch.zeros_like(grid)
            grid_coord[:, 0] = position[0] + (sides[0] / (volume_size - 1)) * grid[:, 0]
            grid_coord[:, 1] = position[1] + (sides[1] / (volume_size - 1)) * grid[:, 1]
            grid_coord[:, 2] = position[2] + (sides[2] / (volume_size - 1)) * grid[:, 2]

            coord_volume = grid_coord.reshape(volume_size, volume_size, volume_size, 3)

            if self.kind == "coco":
                axis = [0, 1, 0]  # y axis
            elif self.kind == "mpii":
                axis = [0, 0, 1]  # z axis

            # random rotation
            if self.training and self.rotation and determine_base_point:
                theta = np.random.uniform(0.0, 2 * np.pi)

                # random rotation at the second stage
                center = torch.from_numpy(base_point).type(torch.float).to(device)
                coord_volume = coord_volume - center
                coord_volume = volumetric.rotate_coord_volume(coord_volume, theta, axis)
                coord_volume = coord_volume + center

            # transfer
            if self.transfer_cmu_to_human36m:
                coord_volume = coord_volume.permute(0, 2, 1, 3)
                inv_idx = torch.arange(coord_volume.shape[1] - 1, -1, -1).long().to(device)
                coord_volume = coord_volume.index_select(1, inv_idx)

            coord_volumes[batch_i] = coord_volume        

        return coord_volumes, cuboids    

In [ ]:
def get_coord_volumes(self, 
                        cuboid_side, 
                        volume_size, 
                        device, 
                        keypoints = None,
                        batch_size = None,
                        dt = None
                        ):
    
    use_origin_basepoint = keypoints is None
    bs_dt = (batch_size, dt) if use_origin_basepoint else keypoints.shape[:-2]
    
    # default base_point is the coordinate's origin
    base_point = np.zeros((*bs_dt, 3))
    
    if not use_origin_basepoint:    
        # get root (pelvis) from keypoints   
        if self.kind == "coco":
            base_point = (keypoints[...,11, :3] + keypoints[...,12, :3]) / 2
        elif self.kind == "mpii":
            base_point = keypoints[..., 6, :3] 

    position = torch.tensor(base_point - sides / 2, dtype=torch.float).to(device)

    # build coord volume
    xxx, yyy, zzz = torch.meshgrid(torch.arange(volume_size, device=device),
                                    torch.arange(volume_size, device=device),
                                     torch.arange(volume_size, device=device))
    grid = torch.stack([xxx, yyy, zzz], dim=-1).type(torch.float)
    grid = grid.view((-1, 3))
    grid = grid.view(*[1]*len(bs_dt), *grid.shape).repeat(*keypoints.shape[:-2], *[1]*len(grid.shape))

    grid[..., 0] = position[..., 0].unsqueeze(-1) + (sides[0] / (volume_size - 1)) * grid[..., 0]
    grid[..., 1] = position[..., 1].unsqueeze(-1) + (sides[1] / (volume_size - 1)) * grid[..., 1]
    grid[..., 2] = position[..., 2].unsqueeze(-1) + (sides[2] / (volume_size - 1)) * grid[..., 2]
    
    if self.kind == "coco":
        axis = [0, 1, 0]  # y axis
    elif self.kind == "mpii":
        axis = [0, 0, 1]  # z axis
        
    # random rotation
    if self.training and self.rotation:    
        
        center = torch.from_numpy(base_point).type(torch.float).to(device).unsqueeze(-2)
        grid = grid - center
        
        grid = torch.stack([volumetric.rotate_coord_volume(coord_grid,\
                            np.random.uniform(0.0, 2 * np.pi), axis) for coord_grid in grid])
        
        grid = grid + center
    
    grid = grid.view(*bs_dt, volume_size, volume_size, volume_size, 3)
    
    return grid

In [ ]:
batch_shorted =  batch.copy()
batch_shorted['keypoints_3d'] = batch_shorted['keypoints_3d'][:,0]

In [ ]:
model = EasyDict({'training': True,
                  'rotation':False,
                  'kind':'mpii',
                  'transfer_cmu_to_human36m':False,
                  'use_precalculated_pelvis': False,
                  'use_gt_pelvis': True,
                  'use_separate_v2v_for_basepoint': False})


In [ ]:
grid_t = []
grid_old_t = []
sizes = list(range(1,40))

for i in tqdm_notebook(sizes):
    
    keypoints = np.random.randn(i,17,4)
    batch_shorted['keypoints_3d'] = keypoints
    
    
    t1 = time()
    grid = get_coord_volumes(model, 
                            cuboid_side, 
                            volume_size, 
                            device, 
                            keypoints = keypoints)

    t2 = time()
    grid_old = get_coord_volumes_old(model, 
                                    batch_shorted, 
                                    cuboid_side, 
                                    volume_size, 
                                    device, 
                                    determine_base_point = True, 
                                    tri_keypoints_3d = None
                                    )

    t3 = time()
    
    grid_t += [round(t2-t1,4)]
    grid_old_t += [round(t3-t2,4)]
    
    assert bool(torch.all(grid == grid_old[0]))

In [ ]:
xticks = sizes
plt.figure(figsize=(10,10))
plt.plot(grid_old_t, label='old_coord_volume_generator')
plt.plot(grid_t, 'r', label='new_coord_volume_generator')
plt.xticks(ticks = range(len(xticks)), labels=xticks)
plt.xlabel('batch_size')
plt.ylabel('time, sec.')
